In [38]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [39]:
 !git clone https://github.com/ratul442/Sentiment-Analysis-Using-Scikit-learn.git

fatal: destination path 'Sentiment-Analysis-Using-Scikit-learn' already exists and is not an empty directory.


In [40]:
df = pd.read_excel('Sentiment-Analysis-Using-Scikit-learn/train.xlsx')

In [41]:
df.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [42]:
!pip install spacy==2.2.3
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [43]:
!pip install git+https://github.com/ratul442/preprocess_kgptalkie.git

  Cloning https://github.com/ratul442/preprocess_kgptalkie.git to /tmp/pip-req-build-ui_ecz98
  Running command git clone -q https://github.com/ratul442/preprocess_kgptalkie.git /tmp/pip-req-build-ui_ecz98
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-cp36-none-any.whl size=11742 sha256=c50c3e847ae38585e28fd00cc256e8d1574a8456c24cfb3da074a5b990f1ee08
  Stored in directory: /tmp/pip-ephem-wheel-cache-sfz9ehaq/wheels/52/f3/03/3bca87b46d7622c5b646fed6a68c12ceb78fe1b5ca8450c85d
Successfully built preprocess-kgptalkie


In [44]:
import preprocess_kgptalkie as ps
import re

In [46]:
def get_clean(x):
    x = str(x).lower().replace('_', ' ').replace('\\', '')

    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\1{2,}", "\1", x)
    return x

df['Reviews'] = df['Reviews'].apply(lambda x: get_clean(x))
df.head()

,Reviews,Sentiment
0,when i first tuned in on this morning news i t...,neg
1,mere thoughts of going overboard aka babes aho...,neg
2,why does this movie fall well below standards ...,neg
3,wow and i thought that any steven segal movie ...,neg
4,the story is seen before but that doesand matt...,neg


In [ ]:
df.head()

In [47]:
tfidf = TfidfVectorizer(max_features=5000)
X = df['Reviews']
y = df['Sentiment']

X = tfidf.fit_transform(X)
X

<25000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2845128 stored elements in Compressed Sparse Row format>

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [49]:
clf = LinearSVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [50]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         neg       0.88      0.87      0.87      2480
         pos       0.87      0.88      0.88      2520

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000



In [51]:
x = 'this movie is really bad. I dislike it'

x = get_clean(x)
vec = tfidf.transform([x])

In [52]:
clf.predict(vec)

array(['neg'], dtype=object)

In [55]:
x = 'Please do not make such kind of movies in future!'

x = get_clean(x)
vec1 = tfidf.transform([x])

In [56]:
clf.predict(vec1)

array(['neg'], dtype=object)

In [58]:
x = 'This movie has a deep connection with my heart, it is truly a masterpiece!'

x = get_clean(x)
vec2 = tfidf.transform([x])

In [59]:
clf.predict(vec2)

array(['pos'], dtype=object)

In [60]:
import pickle
pickle.dump(clf, open('model', 'wb'))
pickle.dump(tfidf, open('tfidf', 'wb'))